In [4]:
import tensorflow_hub as hub
from tensorflow.keras.preprocessing import image
import urllib3  # the lib that handles the url stuff
from PIL import Image
import numpy as np
import os
import csv

In [5]:
detector = hub.load("https://tfhub.dev/tensorflow/centernet/resnet50v1_fpn_512x512/1")

In [6]:
def predict(my_image):
    # commented for open_image_from_google
    # my_image = image.smart_resize(my_image, size=(1024, 1024))
    x = image.img_to_array(my_image)
    x = np.expand_dims(x, axis=0)
    preds = detector(x)
    return preds

In [7]:
def open_image_from_google(coco):

    impath = "D:\\AGH\\VI semestr\\ML\\datasetGoogle\\OIDv4_ToolKit\\OID\\Dataset\\test\\Apple\\964b69442edeec04.jpg"

    apple = Image.open(impath)
    result, x = predict(apple)
    results = {key:value.numpy() for key,value in result.items()}
    detection_classes = results['detection_classes']
    print("model thinks: " + coco[detection_classes[0][0]])

In [8]:
def get_coco_labels():
    http = urllib3.PoolManager()
    r = http.request('GET', 'https://raw.githubusercontent.com/amikelive/coco-labels/master/coco-labels-paper.txt')
    dane = r.data.decode('UTF-8')
    my_data = dane.split('\n')

    coco = {i+1: my_data[i] for i in range(0,len(my_data))}
    return coco


In [9]:
def get_downloadable_labels_str(coco):
    d = {value:1 for value in coco.values()}
    # tmp = d["street sign"]

    # nowa googlowa          stara coco
    d["traffic sign"] = d.pop("street sign")
    d["bull"] = d.pop("cow")
    d["glasses"] = d.pop("eye glasses")
    d["boot"] = d.pop("shoe")
    d["hair dryer"] = d.pop("hair drier")
    d["doughnut"] = d.pop("donut")
    d["microwave oven"] = d.pop("microwave")
    d["telephone"] = d.pop("cell phone")
    d["computer keyboard"] = d.pop("keyboard")
    d["remote control"] = d.pop("remote")
    d["computer mouse"] = d.pop("mouse")
    d["television"] = d.pop("tv")
    d["Kitchen dining room table"] = d.pop("dining table")
    d["plant"] = d.pop("potted plant")
    d["ski"] = d.pop("skis")
    d["volleyball"] = d.pop("sports ball")
    d["coffee cup"] = d.pop("cup")
    d["flying disc"] = d.pop("frisbee")
    # hairbrush
    d.pop("hair brush")

    labels = list(d.keys())
    lb = [x.capitalize() for x in labels]
    lb1 = [label.replace(" ","_") for label in lb]
    str1 = " "
    names = str1.join(lb1)
    print(names)
    return names




In [10]:
def get_paths_to_photos_and_labels(dataset_path="D:\\AGH\\VI semestr\\ML\\datasetGoogle\\OIDv4_ToolKit\\OID\\Dataset\\test"):

    directory_contents = os.listdir(dataset_path)

    class_path_dict = {}

    for subdirectory in directory_contents:
        images = os.listdir(dataset_path + "\\" + subdirectory)

        for image in images:
            if image.endswith(".jpg"):
                full_image_path = dataset_path + "\\" + subdirectory + "\\" + image
                if subdirectory.lower() in class_path_dict:
                    class_path_dict[subdirectory.lower()].append(full_image_path)
                else:
                    class_path_dict[subdirectory.lower()] = [full_image_path]

    return class_path_dict

print(get_paths_to_photos_and_labels())

{'airplane': ['D:\\AGH\\VI semestr\\ML\\datasetGoogle\\OIDv4_ToolKit\\OID\\Dataset\\test\\Airplane\\2e89b6830d277669.jpg'], 'apple': ['D:\\AGH\\VI semestr\\ML\\datasetGoogle\\OIDv4_ToolKit\\OID\\Dataset\\test\\Apple\\44749496c016708d.jpg', 'D:\\AGH\\VI semestr\\ML\\datasetGoogle\\OIDv4_ToolKit\\OID\\Dataset\\test\\Apple\\964b69442edeec04.jpg', 'D:\\AGH\\VI semestr\\ML\\datasetGoogle\\OIDv4_ToolKit\\OID\\Dataset\\test\\Apple\\ba813e858a702b5a.jpg'], 'backpack': ['D:\\AGH\\VI semestr\\ML\\datasetGoogle\\OIDv4_ToolKit\\OID\\Dataset\\test\\Backpack\\723e70fe85e0da69.jpg'], 'banana': ['D:\\AGH\\VI semestr\\ML\\datasetGoogle\\OIDv4_ToolKit\\OID\\Dataset\\test\\Banana\\eeb93d366d6c69e7.jpg'], 'baseball bat': ['D:\\AGH\\VI semestr\\ML\\datasetGoogle\\OIDv4_ToolKit\\OID\\Dataset\\test\\Baseball bat\\9a6747129ee9d39d.jpg'], 'baseball glove': ['D:\\AGH\\VI semestr\\ML\\datasetGoogle\\OIDv4_ToolKit\\OID\\Dataset\\test\\Baseball glove\\0298c70279b6e842.jpg', 'D:\\AGH\\VI semestr\\ML\\datasetGoogle\

In [26]:

def get_all_images_metadata_dict(path="D:\\AGH\\VI semestr\\ML\\datasetGoogle\\OIDv4_ToolKit\\OID\\csv_folder\\test-annotations-bbox.csv"):
    my_dict = {}

    correct_class_id = class_id_to_coco_label_map.keys()

    with open(path, mode='r') as infile:
        reader = csv.reader(infile)
        for row in reader:
            if row[0] not in my_dict:
                if row[2] in correct_class_id:
                    my_dict[row[0]] = []
                    my_dict[row[0]].append(row[1:])
            else:
                if row[2] in correct_class_id:
                    my_dict[row[0]].append(row[1:])




    return my_dict


In [12]:
def get_class_id_to_open_image_label_map(path = "D:\\AGH\\VI semestr\\ML\\datasetGoogle\\OIDv4_ToolKit\\OID\\csv_folder\\class-descriptions-boxable.csv"):
    f = open(path)
    class_map = {}
    for line in f:
        data_line = line.rstrip().split(',')
        class_map[data_line[0]] = data_line[1]
    return class_map

In [13]:
def convert_open_image_label_to_coco(class_map):

    class_map = {value.lower():key for key, value in class_map.items()}

    class_map["street sign"] = class_map.pop("traffic sign")
    class_map["cow"] = class_map.pop("bull")
    class_map["eye glasses"] = class_map.pop("glasses")
    class_map["shoe"] = class_map.pop("boot")
    class_map["hair drier"] = class_map.pop("hair dryer")
    class_map["donut"] = class_map.pop("doughnut")
    class_map["microwave"] = class_map.pop("microwave oven")
    class_map["cell phone"] = class_map.pop("telephone")
    class_map["keyboard"] = class_map.pop("computer keyboard")
    class_map["remote"] = class_map.pop("remote control")
    class_map["mouse"] = class_map.pop("computer mouse")
    class_map["tv"] = class_map.pop("television")
    class_map["dining table"] = class_map.pop("kitchen dining room table")
    class_map["potted plant"] = class_map.pop("plant")
    class_map["skis"] = class_map.pop("ski")
    class_map["sports ball"] = class_map.pop("ball")
    class_map["cup"] = class_map.pop("coffee cup")
    class_map["frisbee"] = class_map.pop("flying disc")
    # hairbrush

    return {value:key for key, value in class_map.items()}

In [30]:
def remove_non_coco_labels(class_id_to_coco_label_map):
    # coco_list
    class_id_to_only_coco_map = {}
    for key,val in class_id_to_coco_label_map.items():
        if val in coco_list:
            class_id_to_only_coco_map[key] = val

    return class_id_to_only_coco_map


In [55]:
def get_image_labels_with_bbox(metadata):
    labes_bboxes = []
    for entry in metadata:
        bbox = [float(i) for i in entry[3:7]]
        labes_bboxes.append([class_id_to_coco_label_map[entry[1]],bbox])

    return labes_bboxes

In [15]:
def predict_single_image(path="D:\\AGH\\VI semestr\\ML\\datasetGoogle\\OIDv4_ToolKit\\OID\\Dataset\\test\\Baseball glove\\0298c70279b6e842.jpg"):
    image_name = os.path.basename(path).split(".")[0]
    image = Image.open(path)
    result = predict(image)
    results = {key:value.numpy() for key,value in result.items()}
    # detection_classes = results['detection_classes']
    # print(results)
    # print("model thinks: " + coco[detection_classes[0][0]])
    # print("model thinks: " + coco[detection_classes[0][1]])
    # print("model thinks: " + coco[detection_classes[0][2]])
    # print("model thinks: " + coco[detection_classes[0][3]])
    # print("accually: ")
    # print(get_image_labels(image_name))
    return results, image_name

In [66]:
def compute_iou(groundtruth_box, detection_box):
    g_xmin, g_xmax, g_ymin, g_ymax = groundtruth_box
    d_ymin, d_xmin, d_ymax, d_xmax = detection_box

    xa = max(g_xmin, d_xmin)
    ya = max(g_ymin, d_ymin)
    xb = min(g_xmax, d_xmax)
    yb = min(g_ymax, d_ymax)

    intersection = max(0, xb - xa + 1) * max(0, yb - ya + 1)

    boxAArea = (g_xmax - g_xmin + 1) * (g_ymax - g_ymin + 1)
    boxBArea = (d_xmax - d_xmin + 1) * (d_ymax - d_ymin + 1)

    return intersection / float(boxAArea + boxBArea - intersection)

In [ ]:
def perform_analysis(model_result, image_name):


    image_metadata_list = all_images_metadata_dict[image_name]
    labels_bboxes_original = get_image_labels_with_bbox(image_metadata_list)


    detected_classes = model_result['detection_classes'][0]
    detected_classes_accuracy = model_result['detection_scores'][0]
    detected_classes_bboxes = model_result['detection_boxes'][0]

    labels_bboxes_detected = []
    for d_class, score, bbox in zip(detected_classes,detected_classes_accuracy, detected_classes_bboxes):
        if score >= 0.5:
            labels_bboxes_detected.append([coco[d_class], list(bbox)])

    print(labels_bboxes_original)
    print(labels_bboxes_detected)
    matches = {}
    for bbox_oryginal in labels_bboxes_original:
        tmp = []
        best_detected_bbox = None
        for bbox_detected in labels_bboxes_detected:
            iou = compute_iou(bbox_oryginal[1], bbox_detected[1])
            if iou >= 0.5:
                tmp.append([iou,bbox_detected])
        if len(tmp) > 0:
            tmp.sort(key = lambda x: x[0])
            best_detected_bbox = tmp[-1][1]

        print(best_detected_bbox)
        print(bbox_oryginal)

        # matches[bbox_oryginal] = tmp[-1]


    # print(detected_classes)
    # print(detected_classes_probability)
    # readable_detected_classes = coco[]






In [56]:
# obtain all needed dicts

class_id_to_open_image_label_map = get_class_id_to_open_image_label_map()
class_id_to_coco_label_map =  convert_open_image_label_to_coco(class_id_to_open_image_label_map)
class_id_to_coco_label_map = remove_non_coco_labels(class_id_to_coco_label_map)

all_images_metadata_dict = get_all_images_metadata_dict()

In [23]:
coco = get_coco_labels()
coco_list  = list(coco.values())

# get_downloadable_labels_str(coco)
# copy returned string
# open in terminal
# python main.py downloader --classes <paste_here_your_string> --type_csv <validation/test/train> --limit XX

In [65]:
results, image_name = predict_single_image()
s = perform_analysis(results, image_name)

[['person', [0.0, 1.0, 0.0, 0.144334]], ['baseball bat', [0.48181, 0.55446, 0.473292, 0.510084]], ['baseball glove', [0.444979, 0.509261, 0.486219, 0.579589]]]
[['person', [0.29661426, 0.22632076, 0.7786821, 0.5014849]], ['person', [0.32117447, 0.44827425, 0.75715, 0.62050307]], ['person', [0.3179868, 0.633381, 0.7627628, 0.82747054]], ['baseball glove', [0.48137105, 0.44447112, 0.565764, 0.5002186]]]
None
['person', [0.0, 1.0, 0.0, 0.144334]]
['baseball glove', [0.48137105, 0.44447112, 0.565764, 0.5002186]]
['baseball bat', [0.48181, 0.55446, 0.473292, 0.510084]]
['baseball glove', [0.48137105, 0.44447112, 0.565764, 0.5002186]]
['baseball glove', [0.444979, 0.509261, 0.486219, 0.579589]]
